#### 제한적 최적화
제한적 최적화는 최적화시 일부 제약조건을 주는 방법으로, 제약조건을 주는 이유는 아래와 같음
* 현실적 제약: 변수가 사람의 나이일 때, 제약조건으로 음수나 200이상을 제거함
* 이 밖에 `안전성과 신뢰성`, `비용 최소화`, `권리와 규정준수` 등의 이유로 제한적 최적화를 할 수 있음

#### 라이브러리 불러오기

In [16]:
import os
import numpy as np
import bayes_opt as bo

import matplotlib.pyplot as plt
from scipy.optimize import NonlinearConstraint
from datetime import datetime as dt

#### 간단한 제약조건이 있는 최적화

##### 비교를 위해 제약조건이 없는 함수 생성

In [3]:
SEED = 0

In [2]:
def black_box_no_constraints(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

In [9]:
pbounds = {'x': (2, 4), 'y': (-3, 3)}

_dt_start = dt.now()
nc_optimizer = bo.BayesianOptimization(
    f=black_box_no_constraints,
    pbounds=pbounds,
    verbose=0,
    random_state=SEED
)

nc_optimizer.maximize(init_points=2, n_iter=100)

print(f'Max values: {nc_optimizer.max}')
print(f'Processing Time: {dt.now()-_dt_start}')

Max values: {'target': -3.000000124830394, 'params': {'x': 2.0, 'y': 0.9996466865497726}}
Processing Time: 0:01:08.632292


##### 제약조건 추가
* 제약조건: y > x

In [5]:
def block_box_with_constraints(x, y):
    return -10 if y <= x else -x ** 2 - (y - 1) ** 2 + 1

In [10]:
_dt_start = dt.now()
wc_optimizer = bo.BayesianOptimization(
    f=block_box_with_constraints,
    pbounds=pbounds,
    verbose=0,
    random_state=SEED
)

wc_optimizer.maximize(init_points=2, n_iter=100)

print(f'Max values: {wc_optimizer.max}')
print(f'Processing Time: {dt.now()-_dt_start}')

Max values: {'target': -4.208073090494202, 'params': {'x': 2.0, 'y': 2.099123783062764}}
Processing Time: 0:00:32.348864


##### 또 다른 제약조건 추가
* 제약조건: x + y = 4 (y = 4 - x)

In [14]:
def surrogate_func(x):
    return black_box_no_constraints(x, 4-x)

pbounds = {'x': (2, 4)}

_dt_start = dt.now()

nc2_optimizer = bo.BayesianOptimization(
    f=surrogate_func,
    pbounds=pbounds,
    allow_duplicate_points=True,
    verbose=0,
    random_state=SEED
)

nc2_optimizer.maximize(init_points=2, n_iter=100)

print(f'Max values: {nc2_optimizer.max}')
print(f'Processing Time: {dt.now()-_dt_start}')

Data point [2.] is not unique. 1 duplicates registered. Continuing ...
Data point [2.] is not unique. 2 duplicates registered. Continuing ...
Data point [2.] is not unique. 3 duplicates registered. Continuing ...
Data point [2.] is not unique. 4 duplicates registered. Continuing ...
Data point [2.] is not unique. 5 duplicates registered. Continuing ...
Data point [2.] is not unique. 6 duplicates registered. Continuing ...
Data point [2.] is not unique. 7 duplicates registered. Continuing ...
Data point [2.] is not unique. 8 duplicates registered. Continuing ...
Data point [2.] is not unique. 9 duplicates registered. Continuing ...
Data point [2.] is not unique. 10 duplicates registered. Continuing ...
Data point [2.] is not unique. 11 duplicates registered. Continuing ...
Data point [2.] is not unique. 12 duplicates registered. Continuing ...
Data point [2.] is not unique. 13 duplicates registered. Continuing ...
Data point [2.] is not unique. 14 duplicates registered. Continuing ...
D

#### 고급 제약조건을 포함한 최적화
* 가우시안 프로세스로 모델링하여 최적화를 구성

##### 간단한 하나의 제약조건

In [17]:
def target_function(x, y):
    return np.cos(2*x) * np.cos(y) + np.sin(x)

def constraint_function(x, y):
    return np.cos(x) * np.cos(y) - np.sin(x) * np.sin(y)

constraint_limit = 0.5

In [18]:
# 여기서부터는 잘 이해가 안되네;;;